In [1]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datasets import load_dataset
import pandas as pd
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




/opt/anaconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dowlnoad the dataset
data_load = load_dataset("nedjmaou/MLMA_hate_speech")
data_train = data_load["train"]
dataset = data_train.to_pandas()
print(dataset.head())
print(dataset.shape)

   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(18661, 7)


In [3]:
#dataset separate by languages
arab_path = "/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/ar_dataset_600.csv"
ar_dataset = pd.read_csv(arab_path)
print(ar_dataset.shape)
print(ar_dataset.head())
ar_dataset = ar_dataset.sample(frac=1, random_state=42)



en_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/en_dataset_600.csv"
en_dataset = pd.read_csv(en_path)
print(en_dataset.shape)
print(en_dataset.head())
en_dataset = en_dataset.sample(frac=1, random_state=42)


fr_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/fr_dataset_600.csv"
fr_dataset = pd.read_csv(fr_path)
print(fr_dataset.shape)
print(fr_dataset.head())
fr_dataset = fr_dataset.sample(frac=1, random_state=42)



(600, 8)
   HITId                                              tweet sentiment  \
0   1486                          بول البعير وعلاج السرطان!    normal   
1   3194  الإسلام يتعامل مع القضايا قبل حصولها فيسعى لعل...    normal   
2   1384  @user @user معلش سؤال..الكائن الابيض دا نوعه ا...    normal   
3   1391  محاضرة التحرش والابتزاز كلية الحقوق والعلوم ال...    normal   
4   1420  حرب شرسه على المرآه وحجابها من كل مكان الاعلام...    normal   

  directness   annotator_sentiment    target  group  sentiment_binary  
0   indirect  indifference_disgust     other  other                 1  
1   indirect          indifference  religion  other                 1  
2   indirect                 shock    origin  other                 1  
3   indirect          indifference     other  other                 1  
4     direct               sadness     other  women                 1  
(600, 8)
   HITId                                              tweet sentiment  \
0   5421                              

In [4]:
#random shuffle
en_dataset = en_dataset.sample(frac=1, random_state=42)


In [5]:
#create binary datasets
en_dataset_binary=en_dataset.copy()
en_dataset_binary = en_dataset_binary[en_dataset_binary['sentiment_binary'].isin([2, 3])]
en_dataset_binary['sentiment_binary'] = en_dataset_binary['sentiment_binary'].replace(2, 0)
en_dataset_binary['sentiment_binary'] = en_dataset_binary['sentiment_binary'].replace(3, 1)

ar_dataset_binary=ar_dataset.copy()
ar_dataset_binary = ar_dataset_binary[ar_dataset_binary['sentiment_binary'].isin([2, 3])]
ar_dataset_binary['sentiment_binary'] = ar_dataset_binary['sentiment_binary'].replace(2, 0)
ar_dataset_binary['sentiment_binary'] = ar_dataset_binary['sentiment_binary'].replace(3, 1)

fr_dataset_binary=fr_dataset.copy()
fr_dataset_binary = fr_dataset_binary[fr_dataset_binary['sentiment_binary'].isin([2, 3])]
fr_dataset_binary['sentiment_binary'] = fr_dataset_binary['sentiment_binary'].replace(2, 0)
fr_dataset_binary['sentiment_binary'] = fr_dataset_binary['sentiment_binary'].replace(3, 1)



In [6]:
#One by One

used_data = en_dataset_binary
predictions_concat=[]

# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = en_dataset['tweet'].iloc[i]
    # prompt
    prompt = [{"role": "system", "content": "Do you think these tweets are hatefull speech or just offensive speech. Try your best even if we do not have context. Display the reasult as an integer of 0 (Offensive) or 1 (Hateful). Do not display anything else. Give an answer of this form even if you are not sure."},
              #{"role": "system", "content": "Do you think these tweets are hatefull speech, offensive or normal speech. Try your best to distinguish the 3 even if we do not have context. Display the reasult as an integer of 1 (Normal), 2(offensive) or 3 (Hateful). Do not display anything else, it is important that only the interger is displayed."},
              {"role": "user", "content": tweet},]
    # LLM initialization
    llm = Ollama(model="llama3")
    predictions = llm.invoke(prompt)
    predictions = int(predictions) 
    predictions_concat.append(predictions)

print(len(predictions_concat))
#add the predictions to the dataset
used_data['predictions'] = predictions_concat


 42%|████▏     | 168/400 [02:04<02:51,  1.35it/s]


ValueError: invalid literal for int() with base 10: 'I cannot determine hatefulness without context. However, based on the tweet alone, I would rate it as 0 (Offensive).'

In [21]:
#Metrics
acc=accuracy_score(used_data['sentiment_binary'], used_data['predictions'])
F1=f1_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Precision=precision_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Recall=recall_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
print(f"Accuracy: {acc:.3f}, F1: {F1:.3f}, Precision: {Precision:.3f}, Recall: {Recall:.3f}")

Accuracy: 0.382, F1: 0.318, Precision: 0.478, Recall: 0.382


In [20]:
print(used_data[[ 'sentiment','sentiment_binary', 'predictions']])

     sentiment  sentiment_binary  predictions
110     normal                 1            1
419    hateful                 3            3
565    hateful                 3            3
77      normal                 1            2
181     normal                 1            3
..         ...               ...          ...
71      normal                 1            3
106     normal                 1            3
270  offensive                 2            3
435    hateful                 3            3
102     normal                 1            3

[600 rows x 3 columns]


In [11]:
print(en_dataset['sentiment_binary'].head())
print(en_dataset.shape)

110    1
419    3
565    3
77     1
181    1
Name: sentiment_binary, dtype: int64
(600, 8)
